In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pycocotools.coco import COCO
from pathlib import Path
from PIL import Image

#import torch
#import torchvision
#from torch.utils.data import Dataset
#from torchvision.transforms import ToTensor
#from torchvision.io import 

### Data Files Exploration

In [2]:
# exploring characteristic csv files
char = pd.read_csv('../Data/all_fields_lincolnbeet/overall_characteristics_dataset.csv')

In [3]:
char.head(13)

,Average items per picture,8.915492957746478
0,Number of items in the dataset,39246.000000
1,Average item density,0.000004
2,Average area occupied by bounding boxes,0.071746
3,Average relative size bounding boxes,0.027236
4,Average percetage of the bounding box that is ...,0.017606
5,number of items class 0,16399.000000
6,Average items per image for class 0,3.725352
7,number of items class 1,22847.000000
8,Average items per image for class 1,5.190141
9,Average image occupation per image for class 0,0.033085


In [4]:
char.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 2 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Average items per picture  13 non-null     object 
 1   8.915492957746478          13 non-null     float64
dtypes: float64(1), object(1)
memory usage: 336.0+ bytes


In [5]:
# loading list of images for training set
train_list = pd.read_json('../Data/all_fields_lincolnbeet/all_fields_lincolnbeet_train_.json')

In [6]:
# exploring train list
train_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3089 entries, 0 to 3088
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3089 non-null   object
dtypes: object(1)
memory usage: 24.3+ KB


In [7]:
# exploring value of train list
train_list[0][0]

'all_fields_lincolnbeet/all/near30_near30_25_05_2021_v_1_96.png'

### COCOJSON File Exploration

In [8]:
# creating COCO object for training set
ann_file_train = Path('../Data/all_fields_lincolnbeet/json_train_set.json')
coco_train = COCO(ann_file_train)

loading annotations into memory...
Done (t=0.11s)
creating index...
index created!


In [31]:
# getting annotations
anns_IDs_train = coco_train.getAnnIds()
anns_train = coco_train.loadAnns(anns_IDs_train)
anns_train[0]

{'area': 22984,
 'bbox': [45, 322, 136, 169],
 'category_id': 0,
 'id': 6,
 'ignore': 0,
 'image_id': 1,
 'iscrowd': 0,
 'segmentation': []}

In [32]:
# getting image IDs
img_IDs_train = coco_train.getImgIds()
imgs_train = coco_train.loadImgs(img_IDs_train)
imgs_train[0]

{'file_name': 'near30_near30_01_06_2021_v_5_70.png',
 'height': 1080,
 'id': 1,
 'width': 1920}

### Loading Data with Pytorch

In [2]:
print(torchvision.__version__)

0.5.0


In [3]:
print(torch.__version__)

1.4.0
